In [25]:
using Revise
using GLMakie
include("EFITModule/EFIT.jl")
include("VoronoiGen.jl")
using BenchmarkTools

In [26]:
using ParallelStencil
using ParallelStencil.FiniteDifferences3D
using GLMakie

ParallelStencil.@reset_parallel_stencil()
USE_GPU = false
@static if USE_GPU
    @init_parallel_stencil(CUDA, Float32, 3)
else
    @init_parallel_stencil(Threads, Float32, 3)
end

In [27]:
nx = ny = nz = 100

nGrains = 1000


#Maximum sound speed in the model
cMax = 8000
cMin = 3000
#Frequency, hz
f0=1e6
#Period
t0 = 1.00 / f0
#Maximum spatial increment
dx = cMin/(8*f0)
#Maximum time increment
dt = dx/(cMax*sqrt(3))

baseMat = Main.EFIT.AnisoMats["X6CrNi1811"]
c = baseMat.c
ρ = baseMat.ρ


7820.0f0

In [28]:
matGrid = fullRandomGen(nx,ny,nz,nGrains);

In [29]:
materials = Array{Main.EFIT.AnisoMat,1}()
randRange = 45
randOff = 22.5
for i in 1:nGrains+2
    rx = rand(Float32)*randRange-randOff
    ry = rand(Float32)*randRange-randOff
    rz = rand(Float32)*randRange-randOff

    mat = Main.EFIT.AnisoMat(ρ, Main.EFIT.rotateMatrix(c,rx,ry,rz))
    push!(materials, mat)
end

In [30]:
grid = Main.EFIT.EFITGrid(Int16.(matGrid.+1),materials,dt,dx);


#Create a source function from which input can be taken
function source(t)
    v = exp(-((2*(t-2*t0)/(t0))^2))*sin(2*pi*f0*t)*0.1
    return Data.Number(v)
end
nframes = 600
framerate = 30

tIterator = 0:grid.dt:grid.dt*nframes
fig,ax,plt = volume(Array(grid.vx),algorithm=:mip,colorrange = (0, 0.1),colormap=:curl, transparency=true)

#Step the simulation
function stepSim(t)
    println(t)
    #Isotropic step function
    Main.EFIT.SimStep!(grid)
    #Apply the source to the Z-direction
    @parallel (45:55,45:55,98:98) Main.EFIT.applySource!(grid.vx,grid.vy,grid.vz, Data.Number(0.0), Data.Number(0.0), (source(t)))

    #Update the plot
    plt.volume = sqrt.(grid.vx.^2 .+ grid.vy.^2 .+ grid.vz.^2)
end

record(stepSim, fig, "color_animation.mp4", tIterator; framerate = 30)


Creating grid of size 100, 100, 100


Initialized averaging dict, hashed 61275 material combinations

0.0
2.7063294e-8
5.4126588e-8
8.118988e-8
1.08253175e-7
1.3531647e-7
1.6237976e-7
1.8944306e-7
2.1650635e-7
2.4356964e-7
2.7063294e-7
2.9769623e-7
3.2475953e-7
3.5182282e-7
3.788861e-7
4.059494e-7
4.330127e-7
4.60076e-7
4.871393e-7
5.142026e-7
5.412659e-7
5.6832914e-7
5.9539246e-7
6.224558e-7
6.4951905e-7
6.765823e-7
7.0364564e-7
7.3070896e-7
7.577722e-7
7.848355e-7
8.118988e-7
8.3896214e-7
8.660254e-7
8.9308867e-7
9.20152e-7
9.472153e-7
9.742786e-7
1.0013418e-6
1.0284052e-6
1.0554685e-6
1.0825318e-6
1.109595e-6
1.1366583e-6
1.1637217e-6
1.1907849e-6
1.2178482e-6
1.2449116e-6
1.2719748e-6
1.2990381e-6
1.3261014e-6
1.3531646e-6
1.380228e-6
1.4072913e-6
1.4343545e-6
1.4614179e-6
1.4884812e-6
1.5155445e-6
1.5426077e-6
1.569671e-6
1.5967344e-6
1.6237976e-6
1.6508609e-6
1.6779243e-6
1.7049875e-6
1.7320508e-6
1.7591141e-6
1.7861773e-6
1.8132407e-6
1.840304e-6
1.8673672e-6
1.8944306e-6
1.921

LoadError: Screen not open!